# Weiner's Attack 2

### Function Definitions

In [88]:
def get_convergents(N,e):
    cf = (e/N).continued_fraction()
    for convergent in cf.convergents():
        yield (convergent.numerator(), convergent.denominator())
        
def get_prime(bitlength):
    while true:
        pr = ZZ.random_element(2**bitlength-1)
        if pr.is_prime():
            break
    return pr

def create_vuln_keypair(bitlength):
    q = get_prime(bitlength // 2)
    while true:
        p = get_prime(bitlength // 2)
        if q < p < 2*q:
            break
    
    N = p*q
    phiN = (p-1)*(q-1)
    
    d_max = int((N**(0.25))/3)
    d = ZZ.random_element(d_max)
    while gcd(phiN, d) != 1:
        d = ZZ.random_element(d_max)
    try:
        e = inverse_mod(d, phiN)
    except:
        print('Modular inverse of (d, phiN) not found')

    assert verify_vulnerability(d, p, q, N)
    assert verify_keypair(N, d, e, 100)
    return N, e, d, p, q, phiN

def verify_vulnerability(d, p, q, N):
    d_constraint = d < int((N**(0.25))/3)
    pq_constraint = q < p < 2*q
    return d_constraint and pq_constraint

def verify_keypair(N, d, e, m):
    c = pow(m,d,N)
    m_ = pow(c,e,N)
    return m == m_

def verify_recovered(N, e, rp, rq, rd):
    rN = rp * rq
    rphiN = (rp-1)*(rq-1)
    re = inverse_mod(rd, rphiN)
    c = pow(100,rd,rN)
    m_ = pow(c,e,N)
    return rN == N and re == e and m_ == 100

def Weiner(N, e):
    convergents = get_convergents(N,e)
    for poss_k, poss_d in convergents:
        if poss_k == 0:
            continue

        poss_phiN = (e*poss_d - 1) // poss_k
        
        x = var('x')
        roots = solve(x^2 - ((N - poss_phiN)+1)*x + N, x, solution_dict=True)
        poss_p = int(roots[0][x])
        poss_q = int(roots[1][x])
        
        if poss_p * poss_q == N:
#             print('p: {}, q: {}, d: {}'.format(poss_p,poss_q,poss_d))
            return poss_p, poss_q, poss_d



### Test Weiner attack with random vulnerable keypairs

In [91]:
N, e, d, p, q, phiN = create_vuln_keypair(128)

poss_p, poss_q, poss_d = Weiner(N, e)

print('Generated p: {}\nGenerated q: {}\nGenerated d: {}'.format(p, q, d))
print('-----------------------')
print('Recovered p: {}\nRecovered q: {}\nRecovered d: {}'.format(poss_p,poss_q,poss_d))

Generated p: 690765036399366823
Generated q: 681929726233972327
Generated d: 240609323
-----------------------
Recovered p: 681929726233972327
Recovered q: 690765036399366823
Recovered d: 240609323


### Crack Group 7's keypair

In [92]:
# Group 7's (N, e) pair
N_big = 947915625495728052916879147697360002848381185192346505887337318484424613804479379229500825154943023456531602566274207306369591959850608246425659384399592991634207719975226030904151882757589795301726622205405989011351842286767545223253169802897983637243167859583565776182244623125858428782210670308167292094521748561446916209557796073456184675807739356933731724960127222046486932110371797638731014250317825447454900437565086817233575744240640382684548268127956137173595899977563370261355506081974467333993545072663993466761303977372442140485495352997067642695566437175979143720808011678287958025730556920566713423095016601310050215390374360676665989625828596980532017231388237244123248528625713956773078115159965411732942413787894237594334962280338475577955633910836559629956564026617798854751095370708381339497715897167336403274224664697600147738325680779222350682292639751395534249345846969763993219058901236669322261406103455020265135135221755871135240191771715762695473152226592913073165991509715785601507547540424028508272006742607644751844712565734384457757831906046716752425927716813761510281226714973672364905890290774159582494385327020386478310127596750245584058550668842273035173930485514220124363571210008854397877759878109
e_big = 704547874817628536678711224260347170080780157225302414633058087631273478699354942316191545183552970491256712479930884728272203750707194999236141913284914475190760565306955811426771755057311324861065120995469067811477070684340373917672377871037102399678405113518962175705845425490703244686038865941492819081360216141451957694835143605736374176464955479837962508015825872034637667723449908810156455398586426946187983417091803239944119556424937448702560147449296000358404512385942224137056969472773149280435494014441168305623041946847983160401839626890022631063357714302185156647516439105432586919947268965890950159670021155797827170793898880619668912414078180761247776607171716089112584888939756383508383552798449319245738264857608637364581842457448498549984823842222174304406071636079197079103741311287709390835892500282167064692991655929368336712543591994988841130885756415329962100690973384132459114994258583928548266879551395010165988053756662119802587150754631529097435381252816428859268931378932500218048710530873387464697995565066027920879230108859236321171012617250021403409639363576159429016313582199500594524217662031829176343714208316657171662175877320736019254117219869310765030193442816632528910296054592307278119395320563

recovered_p = 29741469921397510399335846987018913004297391468646316128211885689651671688643707996145955483148504570570967814114802673427400304193298345737232712434679599743712299481679713707977113163426098260275753085934132989905872434133679404599977307277800206859005292357167013333946644089503208696590370265677420052952325454118739174320696161582809114820851913069046285801337682449696306538690023739084986448815095289913975830598028165668463095391574736362019873695201395665100930661964333423223812375750934839872181229057115071408967412539669136146467898869900806033672052125847919668646035398983509530704930469052562454624931
recovered_q = 31871848567032318794631861601708553572242202430707558768931714326854741650166362886718454726379891564674552365855918421093255093137819182963809939861847256920754971985578303496170842718293867434012375462617800834448543766943581990856807085340488994126703550859956895250011826442960164890577683343613080469685996786642207886204193256576907872921043921086713732197607389695742344682872052881692398204634821940228482861945360949338494513119090742543622579839277184507786680200483276527941055935586032948632750229592510494300913231297570045476708186952534516178388488239540196579767558951459361085474169453298887355731839
recovered_d = 28101755690428597260254340950908534958141005105342900038979086265712806129666367728328780706429667440554160007350114400316333712327597211970150330139322098349623861855862535086434376857048896477880980193640181176018858519878537122636212057363411076026296686449249830386451299272706513121215369767854921864047

# poss_p, poss_q, poss_d = Weiner(N_big, e_big)
# Output:
# p: 29741469921397510399335846987018913004297391468646316128211885689651671688643707996145955483148504570570967814114802673427400304193298345737232712434679599743712299481679713707977113163426098260275753085934132989905872434133679404599977307277800206859005292357167013333946644089503208696590370265677420052952325454118739174320696161582809114820851913069046285801337682449696306538690023739084986448815095289913975830598028165668463095391574736362019873695201395665100930661964333423223812375750934839872181229057115071408967412539669136146467898869900806033672052125847919668646035398983509530704930469052562454624931
# q: 31871848567032318794631861601708553572242202430707558768931714326854741650166362886718454726379891564674552365855918421093255093137819182963809939861847256920754971985578303496170842718293867434012375462617800834448543766943581990856807085340488994126703550859956895250011826442960164890577683343613080469685996786642207886204193256576907872921043921086713732197607389695742344682872052881692398204634821940228482861945360949338494513119090742543622579839277184507786680200483276527941055935586032948632750229592510494300913231297570045476708186952534516178388488239540196579767558951459361085474169453298887355731839
# d: 28101755690428597260254340950908534958141005105342900038979086265712806129666367728328780706429667440554160007350114400316333712327597211970150330139322098349623861855862535086434376857048896477880980193640181176018858519878537122636212057363411076026296686449249830386451299272706513121215369767854921864047

In [93]:
cracked = verify_recovered(N_big, e_big, recovered_p, recovered_q, recovered_d)
print('Group 7 keypair cracked? ', cracked)

Group 7 keypair cracked?  True
